In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install ctgan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from ctgan import CTGAN
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트 1/MBA.csv')

In [6]:

# 'race' 컬럼에서 결측치를 'unknown'으로 채움
df['race'] = df['race'].fillna('unknown')

# 'admission' 컬럼에서 결측치를 'denied'로 채움
df['admission'] = df['admission'].fillna('denied')

# 'admission' 컬럼에서 'Waitlist' 값을 'denied'로 변환
df['admission'] = df['admission'].apply(lambda x : 'Admit' if x == 'Waitlist' else x)

# 'admission' 컬럼을 이진 변수로 변환 ('denied' = 1, 그 외 = 0)
df['admission'] = df['admission'].apply(lambda x : 1 if x == 'denied' else 0)

# 'application_id' 컬럼은 필요 없으므로 제거
df = df.drop('application_id', axis = 1)

# 데이터 준비 (admission 데이터 준비 완료)
# X와 y는 이미 준비되어 있고, 소수 클래스(0)에 대해 증강하려고 함

# 소수 클래스만 추출 (여기서는 '0'이 소수 클래스라고 가정)
minority_class_data = df[df['admission'] == 0]  # admission이 0인 데이터 추출
minority_class_data = minority_class_data.drop(columns=['admission'])  # 타겟 열 제외

# CTGAN 모델 정의 및 학습
ctgan = CTGAN(epochs=100)  # 학습할 epoch 수는 필요에 따라 조정 가능
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
ctgan.fit(minority_class_data, discrete_columns=categorical_cols)  # 범주형 컬럼 명시

# 새로운 데이터 생성 (예: 4800개의 소수 클래스 데이터 생성)
new_samples = ctgan.sample(4800)

# 생성된 데이터를 원래 소수 클래스의 레이블인 0으로 지정
new_samples['admission'] = 0

# 기존 데이터와 결합
df_augmented = pd.concat([df, new_samples], ignore_index=True)

# LabelEncoder 객체 생성
le = LabelEncoder()

# 범주형 컬럼(object 타입)을 모두 찾아서 레이블 인코딩 수행
categorical_cols = df_augmented.select_dtypes(include=['object']).columns

# 각 범주형 컬럼에 Label Encoding 적용
for col in categorical_cols:
    df_augmented[col] = le.fit_transform(df_augmented[col])  # 인코딩 적용
    print(f"Classes (범주) for {col}: {le.classes_}")  # 원래 범주들 출력
    encoded_data = df_augmented[col]  # 인코딩된 값을 데이터프레임에 저장
    mapping = dict(zip(le.classes_, range(len(le.classes_))))  # 범주와 인코딩 값 매핑
    print(f"Mapping (범주 -> 인코딩 값) for {col}: {mapping}\n")

# X와 y 분리 (증강된 데이터로)
X = df_augmented.drop('admission', axis=1)
y = df_augmented['admission']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled_augmented = scaler.fit_transform(X)

# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled_augmented, y, test_size=0.2, random_state=42)

Classes (범주) for gender: ['Female' 'Male']
Mapping (범주 -> 인코딩 값) for gender: {'Female': 0, 'Male': 1}

Classes (범주) for major: ['Business' 'Humanities' 'STEM']
Mapping (범주 -> 인코딩 값) for major: {'Business': 0, 'Humanities': 1, 'STEM': 2}

Classes (범주) for race: ['Asian' 'Black' 'Hispanic' 'Other' 'White' 'unknown']
Mapping (범주 -> 인코딩 값) for race: {'Asian': 0, 'Black': 1, 'Hispanic': 2, 'Other': 3, 'White': 4, 'unknown': 5}

Classes (범주) for work_industry: ['CPG' 'Consulting' 'Energy' 'Financial Services' 'Health Care'
 'Investment Banking' 'Investment Management' 'Media/Entertainment'
 'Nonprofit/Gov' 'Other' 'PE/VC' 'Real Estate' 'Retail' 'Technology']
Mapping (범주 -> 인코딩 값) for work_industry: {'CPG': 0, 'Consulting': 1, 'Energy': 2, 'Financial Services': 3, 'Health Care': 4, 'Investment Banking': 5, 'Investment Management': 6, 'Media/Entertainment': 7, 'Nonprofit/Gov': 8, 'Other': 9, 'PE/VC': 10, 'Real Estate': 11, 'Retail': 12, 'Technology': 13}



In [7]:
from scipy.stats import chi2_contingency

selected_columns = []

for element in df_augmented.columns:

    contingency_table = pd.crosstab(df_augmented[element], df_augmented['admission'])

    # 카이제곱 검정 수행
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    if p < 0.03:
        selected_columns.append(element)

    print(element)
    print("Chi-Square Statistic:", chi2)
    print("p-value:", p)
    print("Degrees of Freedom:", dof)
    print("")

gender
Chi-Square Statistic: 377.2231983696257
p-value: 5.006340162987034e-84
Degrees of Freedom: 1

international
Chi-Square Statistic: 3923.3466669190448
p-value: 0.0
Degrees of Freedom: 1

gpa
Chi-Square Statistic: 7997.3276775635995
p-value: 5.7973130438393324e-154
Degrees of Freedom: 4900

major
Chi-Square Statistic: 11.64232329602769
p-value: 0.002964159833665473
Degrees of Freedom: 2

race
Chi-Square Statistic: 107.61564090398291
p-value: 1.3068988789324653e-21
Degrees of Freedom: 5

gmat
Chi-Square Statistic: 7364.132618401951
p-value: 0.0
Degrees of Freedom: 226

work_exp
Chi-Square Statistic: 597.3232854827723
p-value: 8.803459285945988e-124
Degrees of Freedom: 8

work_industry
Chi-Square Statistic: 130.39965861955227
p-value: 1.741152599233259e-21
Degrees of Freedom: 13

admission
Chi-Square Statistic: 10989.988175730125
p-value: 0.0
Degrees of Freedom: 1



In [8]:
# 랜덤 포레스트 모델 학습
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 특성 중요도 추출
importances = model.feature_importances_
print(len(importances))
# 특성 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# 중요도 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(feature_importance_df)


8
         Feature  Importance
5           gmat    0.308437
2            gpa    0.266490
1  international    0.223397
4           race    0.067239
7  work_industry    0.054232
6       work_exp    0.038474
3          major    0.022902
0         gender    0.018828


In [9]:
features = []
remaining_features = list(range(X.shape[1]))
best_score = 0

while remaining_features:
    scores = []
    for feature in remaining_features:
        # 현재 선택된 특성 + 새 특성
        selected_features = features + [feature]
        model = LogisticRegression(max_iter=200)
        model.fit(X_train[:, selected_features], y_train)
        predictions = model.predict(X_test[:, selected_features])
        score = f1_score(y_test, predictions)
        scores.append((score, feature))

    # 가장 성능이 좋은 특성 선택
    scores.sort(reverse=True)
    best_new_score, best_new_feature = scores[0]

    if best_new_score > best_score:
        best_score = best_new_score
        features.append(best_new_feature)
        remaining_features.remove(best_new_feature)
    else:
        break


print(f"최고 높은 점수: {best_score}")
for element in features:
    print(f"선택된 특성: {df.columns[element]}")

최고 높은 점수: 0.8725536992840095
선택된 특성: gmat
선택된 특성: international
선택된 특성: gpa
선택된 특성: race


In [10]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)

lasso_coef = pd.Series(lasso.coef_, index=X.columns)
selected_features = lasso_coef[lasso_coef != 0].index

print(f"Selected Features by Lasso: {selected_features}")


Selected Features by Lasso: Index(['gender', 'international', 'gpa', 'race', 'gmat', 'work_exp'], dtype='object')


In [11]:
df.columns

Index(['gender', 'international', 'gpa', 'major', 'race', 'gmat', 'work_exp',
       'work_industry', 'admission'],
      dtype='object')

# 선택된 특성

# gmat
# gpa
# work_exp

In [16]:
# X = df_augmented.drop(['admission', 'major', 'gender', 'race', 'international', 'work_industry'], axis=1)
X = df_augmented.drop('admission', axis=1)
y = df_augmented['admission']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled_augmented = scaler.fit_transform(X)

# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled_augmented, y, test_size=0.2, random_state=42)

In [17]:
models = [
    ("Logistic Regression", LogisticRegression(class_weight='balanced')),
    ("K-Nearest Neighbors", KNeighborsClassifier(n_neighbors=3)),
    ("Decision Tree", DecisionTreeClassifier(class_weight='balanced', random_state=42)),
    ("Random Forest", RandomForestClassifier(class_weight='balanced', random_state=42)),
    ("Support Vector Machine", SVC(probability=True)),
    ("XGBOOST", XGBClassifier(scale_pos_weight=10, random_state=42)),
    ("GaussianNB", GaussianNB()),
    ("GradientBoostingClassifier", GradientBoostingClassifier()),
    ("AdaBoostClassifier", AdaBoostClassifier())
]

for model_name, model in models:
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    if hasattr(model, "predict_proba"):
        y_probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_probs)
        print(f"ROC-AUC: {roc_auc:.2f}")


Model: Logistic Regression
Accuracy: 87.90%
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      1152
           1       0.87      0.88      0.87      1047

    accuracy                           0.88      2199
   macro avg       0.88      0.88      0.88      2199
weighted avg       0.88      0.88      0.88      2199

ROC-AUC: 0.95

Model: K-Nearest Neighbors
Accuracy: 86.86%
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      1152
           1       0.85      0.88      0.86      1047

    accuracy                           0.87      2199
   macro avg       0.87      0.87      0.87      2199
weighted avg       0.87      0.87      0.87      2199

ROC-AUC: 0.92

Model: Decision Tree
Accuracy: 85.95%
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1152
           

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Model: AdaBoostClassifier
Accuracy: 88.63%
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1152
           1       0.87      0.89      0.88      1047

    accuracy                           0.89      2199
   macro avg       0.89      0.89      0.89      2199
weighted avg       0.89      0.89      0.89      2199

ROC-AUC: 0.96
